In [ ]:
import os
import re
import numpy as np
import pandas as pd
import cv2
import imageio
import xml.etree.ElementTree as ET

import czifile
import tifffile as tiff

from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, binary_erosion
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label
from skimage.segmentation import watershed
from scipy.ndimage import binary_fill_holes
import napari

# ===============================
# USER CONFIGURATION
# ===============================
# Path to your file (.tif or .czi)
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.tif"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/40A_UAS-TMEM192-3xHA x 40A 71G10 MARCM_around 12h - for quantification_3 Airy-CBs_300425.tif"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/TMEM-HA 6h CB airy_3_170722.tif"

# Correct microscope calibration (µm per pixel)
DEFAULT_VX_VY_UM = 0.04   # <-- YOUR TRUE XY SCALE
MAX_REASONABLE_VXY_UM = 0.5  # µm/px sanity limit

# ---------------------------
# Helper: refine radii by distance transform
# ---------------------------
def refine_radii_via_dt(img3d, blobs, win_px=25, bin_method="sauvola"):
    from skimage.filters import threshold_sauvola, threshold_local, threshold_otsu
    from skimage.morphology import remove_small_objects, disk, binary_opening
    from scipy.ndimage import distance_transform_edt as _edt
    from skimage.measure import label as _label

    if blobs is None or len(blobs) == 0:
        return blobs

    Z, H, W = img3d.shape
    out = blobs.copy().astype(np.float32)

    for i, (zc, yc, xc, _) in enumerate(out):
        z, y, x = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= z < Z and 0 <= y < H and 0 <= x < W):
            continue

        y1, y2 = max(0, y - win_px), min(H, y + win_px + 1)
        x1, x2 = max(0, x - win_px), min(W, x + win_px + 1)
        patch = img3d[z, y1:y2, x1:x2]

        if bin_method == "sauvola":
            ws = max(21, 2*(win_px//2)+1)
            thr = threshold_sauvola(patch, window_size=ws, k=0.2)
            bw = patch > thr
        elif bin_method == "local":
            ws = max(21, 2*(win_px//2)+1)
            thr = threshold_local(patch, block_size=ws, offset=-0.2*np.std(patch))
            bw = patch > thr
        else:
            from skimage.filters import threshold_otsu
            try:
                thr = threshold_otsu(patch)
                bw = patch > thr
            except ValueError:
                continue

        bw = binary_opening(bw, footprint=disk(1))
        bw = remove_small_objects(bw, min_size=3, connectivity=2)

        yy, xx = y - y1, x - x1
        if not (0 <= yy < bw.shape[0] and 0 <= xx < bw.shape[1]) or not bw[yy, xx]:
            continue

        lab = _label(bw)
        lbl = lab[yy, xx]
        if lbl == 0:
            continue
        bw_obj = (lab == lbl)
        dt = _edt(bw_obj)
        r_px = float(dt[yy, xx])
        if r_px <= 0:
            continue
        out[i, 3] = r_px

    return out

# ===============================
# Image loading and metadata
# ===============================
def _try_float(x):
    try:
        return float(x)
    except:
        return None

def _parse_ome_xml(xml_text):
    if not xml_text:
        return None
    def grab(attr):
        m = re.search(fr'PhysicalSize{attr}="([\d\.eE+-]+)"', xml_text)
        return float(m.group(1)) if m else None
    return grab("X"), grab("Y"), grab("Z")

def load_any(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext in (".tif", ".tiff"):
        with tiff.TiffFile(file_path) as tf:
            arr = tf.asarray()
            try:
                ome_xml = tf.ome_metadata
            except Exception:
                ome_xml = None
            vx_um = vy_um = vz_um = None
            if ome_xml:
                vx_um, vy_um, vz_um = _parse_ome_xml(ome_xml)
        img = np.squeeze(arr)
        if img.ndim == 4:
            ch1, ch2 = (img[0], img[1]) if img.shape[0] == 2 else (img[:, 0], img[:, 1])
        else:
            raise RuntimeError("Unexpected TIFF shape")
        return ch1, ch2, (vx_um, vy_um, vz_um), {"type": "tiff"}
    else:
        raise ValueError("Unsupported file format")

# Load image and scales
img_ch1, img_ch2, (vx_um, vy_um, vz_um), meta = load_any(file_path)
if vx_um is None or vy_um is None:
    vx_um = vy_um = DEFAULT_VX_VY_UM
if vz_um is None:
    vz_um = vx_um

if vx_um > MAX_REASONABLE_VXY_UM:
    raise ValueError(f"XY pixel size too large: {vx_um} µm/px")

px_um_xy = float(np.sqrt(vx_um * vy_um))
voxel_um3 = vx_um * vy_um * vz_um
print(f"Voxel size (µm): X={vx_um}, Y={vy_um}, Z={vz_um}")

# ===== Aliases used throughout (fixes NameError) =====
image   = img_ch1       # lysosome channel
image_2 = img_ch2       # neuron / context channel

# ===============================
# Lysosome detection and scaling
# ===============================
image_smooth = gaussian(image, sigma=1.5)

blobs = blob_log(image_smooth, min_sigma=0.8, max_sigma=3.0, num_sigma=4, threshold=0.003)
if len(blobs) > 0:
    blobs[:, 3] *= np.sqrt(3)

blobs = refine_radii_via_dt(image_smooth, blobs)
if len(blobs) > 0:
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um
    radius_um = blobs[:, 3] * px_um_xy
    diameter_um = 2 * radius_um
    volume_um3 = (4/3) * np.pi * radius_um**3
else:
    z_um = y_um = x_um = radius_um = diameter_um = volume_um3 = np.array([])

df = pd.DataFrame({
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "radius_um": radius_um,
    "diameter_um": diameter_um,
    "volume_um3": volume_um3,
})
df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv")

# ===============================
# Napari visualization
# ===============================
viewer = napari.Viewer()
viewer.add_image(img_ch2, name="Ch2 raw")
viewer.add_image(img_ch1, name="Ch1 raw")
viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.7)#0.35

# ==========================================
# 4) CH2: segmentation (CELL vs OUTSIDE)
# ==========================================
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=1.5, preserve_range=True)

# Local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=301, offset=-0.2*np.std(R))
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=20000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(10))
neuron_mask = binary_erosion(neuron_mask, ball(7))

# Soma via distance + cleanup
dist = edt(neuron_mask)
cell_min_radius_vox = 1
cell_mask = (dist >= cell_min_radius_vox)
cell_mask &= neuron_mask
cell_mask = binary_opening(cell_mask, ball(5))
cell_mask = binary_closing(cell_mask, ball(10))
cell_mask = binary_fill_holes(cell_mask)

# Label and territories
body_lab = label(cell_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cells (soma).")

if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels:", int(cell_mask.sum()))

# ==========================================
# ADD-ON: optional filter out oversized cell bodies
# ==========================================
#MAX_BODY_VOXELS = 20000  # set >0 to enable filtering
try:
    if isinstance(cell_seg, np.ndarray) and cell_seg.max() > 0 and MAX_BODY_VOXELS > 0:
        counts = np.bincount(cell_seg.ravel().astype(np.int64))
        drop_labels = np.where(counts < 150000)[0]
        #drop_labels = np.where((counts < 70000) | (counts > 140000))[0]
        drop_labels = drop_labels[drop_labels != 0]

        if drop_labels.size > 0:
            print(f"Filtered {len(drop_labels)} oversized cells. IDs: {drop_labels.tolist()}")
            to_remove = np.isin(cell_seg, drop_labels)
            cell_seg[to_remove] = 0
            if 'cell_mask' in globals() and isinstance(cell_mask, np.ndarray):
                cell_mask[to_remove] = False

            pd.DataFrame({
                "filtered_cell_id_ch2": drop_labels,
                "voxel_count": counts[drop_labels]
            }).to_csv("filtered_cells_gt_threshold.csv", index=False)
        else:
            print(f"No cells exceeded {MAX_BODY_VOXELS} voxels; nothing filtered.")
except Exception as e:
    print("Cell size filtering failed:", e)

# ==========================================
# 5) Map lysosomes to (cell/outside) with per-cell IDs
# ==========================================
location_ch2 = []
cell_id_list = []

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue
        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv (µm-only metrics)")

# Include zero-count cells
try:
    if isinstance(cell_seg, np.ndarray) and cell_seg.max() > 0:
        all_cells = pd.DataFrame({
            "cell_id_ch2": np.arange(1, int(cell_seg.max()) + 1, dtype=int)
        })

        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts_nonzero = (
                df[df["location_ch2"] == "cell"]
                .groupby("cell_id_ch2")
                .size()
                .reset_index(name="count")
            )
        else:
            lys_counts_nonzero = pd.DataFrame(columns=["cell_id_ch2", "count"])

        lys_counts_all = (
            all_cells.merge(lys_counts_nonzero, on="cell_id_ch2", how="left")
                     .fillna({"count": 0})
        )
        lys_counts_all["count"] = lys_counts_all["count"].astype(int)

        lys_counts_all.to_csv("lysosome_counts_by_cell.csv", index=False)
        print("Updated: lysosome_counts_by_cell.csv now includes cells with 0 lysosomes.")
except Exception as e:
    print("Could not expand lysosome_counts_by_cell with zero-count cells:", e)

# ==========================================
# 5b) Per-cell (Ch2) volumes (µm^3)
# ==========================================
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if isinstance(cell_seg, np.ndarray) and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Merge volume with lysosome counts per cell
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# ==========================================
# 6) Visualization (CELL vs OUTSIDE only)
# ==========================================
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'

    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Lysosome points overlay (show ONLY lysosomes inside cells)
if len(blobs) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    in_cell_mask = (df["location_ch2"].to_numpy() == "cell")
    if in_cell_mask.any():
        blobs_cell = blobs[in_cell_mask, :3]            # z,y,x
        radii_cell = blobs[in_cell_mask, 3] if blobs.shape[1] > 3 else np.ones(np.count_nonzero(in_cell_mask))
        pts = viewer.add_points(
            blobs_cell,
            size=np.clip(radii_cell * 2, 2, None),
            name='Lysosomes (cell only)'
        )
        try:
            pts.face_color = [0.0, 1.0, 1.0, 1.0]       # cyan
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df.loc[in_cell_mask, 'id'].to_numpy(),
                'cell':   df.loc[in_cell_mask, 'cell_id_ch2'].to_numpy(),
                'diameter_um': df.loc[in_cell_mask, 'diameter_um'].to_numpy(),
                'volume_um3':  df.loc[in_cell_mask, 'volume_um3'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
        except Exception:
            pass

# ==========================================
# 7) Quick fused 2D video (optional)
# ==========================================

img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)
    
    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        # iterate only over blobs that are inside the cell on this slice
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                # draw only in-cell lysosomes (cyan/yellow are both OK; keeping yellow for visibility)
                cv2.circle(overlay, (x, y), r, (255, 255, 0), 2)
        # NOTE: no drawing at all for outside points
    
    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# 8) Per-cell rotation videos (X, Y, Z axes) – only for cells with lysosomes
# ==========================================

from scipy.ndimage import rotate as ndrotate

def _to_uint8(vol):
    v = vol.astype(np.float32)
    vmin, vmax = float(v.min()), float(v.max())
    if vmax <= vmin:
        return np.zeros_like(v, dtype=np.uint8)
    return (255.0 * (v - vmin) / (vmax - vmin)).astype(np.uint8)

def _rotate_points(pts_zyx, center_zyx, angle_deg, axis="Y"):
    #Rotate 3D points around chosen axis (X, Y, or Z).
    if pts_zyx.size == 0:
        return pts_zyx
    theta = np.deg2rad(angle_deg)
    c, s = np.cos(theta), np.sin(theta)
    p = pts_zyx - center_zyx
    if axis.upper() == "Y":     # rotate in ZX plane
        z = p[:, 0]*c + p[:, 2]*s
        y = p[:, 1]
        x = -p[:, 0]*s + p[:, 2]*c
    elif axis.upper() == "X":   # rotate in YZ plane
        z = p[:, 0]
        y = p[:, 1]*c - p[:, 2]*s
        x = p[:, 1]*s + p[:, 2]*c
    else:                       # Z axis (rotate in XY plane)
        z = p[:, 0]
        y = p[:, 1]*c + p[:, 2]*s
        x = -p[:, 1]*s + p[:, 2]*c
    return np.stack([z, y, x], axis=1) + center_zyx

def save_cell_spin_videos_allaxes(cell_labels, blobs, df,
                                  out_dir="cell_spin_videos",
                                  angles=tuple(range(0, 360, 10)),
                                  pad=8, fps=12,
                                  draw_point_radius_px=3):
    
    #Create per-cell rotation videos around X, Y, and Z axes.
    #Only exports cells that contain lysosomes.
    #Shows:
    #  - Green cell footprint
    #  - Cyan lysosome points inside cell
    #  - Cell ID text
    
    if not isinstance(cell_labels, np.ndarray) or cell_labels.max() == 0:
        print("No cells to export.")
        return
    if not isinstance(df, pd.DataFrame) or "cell_id_ch2" not in df.columns:
        print("Missing per-blob metadata; cannot filter by lysosome presence.")
        return

    cell_ids_with_lys = sorted(df.loc[
        (df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0),
        "cell_id_ch2"
    ].unique().astype(int))
    if len(cell_ids_with_lys) == 0:
        print("No cells contain lysosomes; no videos exported.")
        return

    os.makedirs(out_dir, exist_ok=True)
    COL_CYAN, COL_GREEN, TXT_COL = (255, 255, 0), (0, 255, 0), (240, 240, 240)
    props = [p for p in regionprops(cell_labels) if p.label in cell_ids_with_lys]
    axes_all = ["X", "Y", "Z"]
    Ztot, Ytot, Xtot = cell_labels.shape
    exported = 0

    for p in props:
        cid = p.label
        minz, miny, minx, maxz, maxy, maxx = p.bbox
        miny, maxy = max(0, miny - pad), min(Ytot, maxy + pad)
        minx, maxx = max(0, minx - pad), min(Xtot, maxx + pad)
        sub_lab = cell_labels[minz:maxz, miny:maxy, minx:maxx]
        if (sub_lab == cid).sum() == 0:
            continue
        subZ, subY, subX = sub_lab.shape
        center_zyx = np.array([(subZ-1)/2, (subY-1)/2, (subX-1)/2], dtype=np.float32)

        # lysosomes in this cell
        sub_blobs = df[(df["cell_id_ch2"] == cid) & (df["location_ch2"] == "cell")]
        if sub_blobs.empty:
            continue
        sel = sub_blobs.index.to_numpy()
        pts_local = np.stack([
            blobs[sel, 0]-minz,
            blobs[sel, 1]-miny,
            blobs[sel, 2]-minx
        ], axis=1).astype(np.float32)

        # iterate over each axis type
        for axis in axes_all:
            frames = []
            for ang in angles:
                # rotate volume around chosen axis
                if axis == "Y":
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(0,2), reshape=False, order=0, mode='nearest')
                elif axis == "X":
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(1,0), reshape=False, order=0, mode='nearest')
                else:  # Z
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(1,2), reshape=False, order=0, mode='nearest')

                mask_rot = (rot_lab > 0)
                footprint = mask_rot.max(axis=0).astype(np.uint8)
                frame = np.zeros((subY, subX, 3), dtype=np.uint8)

                # green overlay
                if footprint.any():
                    green = (footprint * 255)
                    overlay = frame.copy()
                    overlay[..., 1] = np.maximum(overlay[..., 1], green)
                    frame = cv2.addWeighted(frame, 1.0, overlay, 0.35, 0.0)
                    contours, _ = cv2.findContours((footprint*255), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    if contours:
                        cv2.drawContours(frame, contours, -1, COL_GREEN, 1, lineType=cv2.LINE_AA)

                #cv2.putText(frame, f"Cell {cid}  {axis}-rot {ang:03d}°", (8, 18),
                #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, TXT_COL, 1, cv2.LINE_AA)

                # rotate and draw lysosome points
                pts_rot = _rotate_points(pts_local, center_zyx, ang, axis=axis)
                ys = np.clip(np.round(pts_rot[:, 1]).astype(int), 0, subY-1)
                xs = np.clip(np.round(pts_rot[:, 2]).astype(int), 0, subX-1)
                for k in range(len(xs)):
                    cv2.circle(frame, (int(xs[k]), int(ys[k])), draw_point_radius_px, COL_CYAN, -1, lineType=cv2.LINE_AA)

                frames.append(frame)

            out_path = os.path.join(out_dir, f"cell_{cid:03d}_spin{axis}.mp4")
            try:
                imageio.mimsave(out_path,
                                [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames],
                                fps=fps, format='FFMPEG')
                print(f"Saved: {out_path}")
            except TypeError:
                out_gif = out_path.replace(".mp4", ".gif")
                imageio.mimsave(out_gif,
                                [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames],
                                fps=fps)
                print(f"Saved (GIF): {out_gif}")
            exported += 1

    print(f"Exported {exported} per-cell rotation videos (X, Y, Z) → {out_dir}")


# ---- call it ----
save_cell_spin_videos_allaxes(cell_seg, blobs, df,
                              out_dir="cell_spin_videos",
                              angles=tuple(range(0, 360, 10)),
                              pad=8, fps=12)

# ==========================================
# 9) 3D screenshots (XY, YZ, XZ)
# ==========================================
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

viewer.dims.ndisplay = 3
save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# ==========================================
# 10) EXTRA: Export Original stack (Ch1) as MP4
# ==========================================
try:
    raw_stack = np.array(image, dtype=np.float32)
    raw_norm = (255 * (raw_stack - raw_stack.min()) / (raw_stack.ptp() + 1e-8)).astype(np.uint8)

    frames_raw = []
    for z in range(raw_norm.shape[0]):
        frame_gray = raw_norm[z]
        frame_bgr = cv2.cvtColor(frame_gray, cv2.COLOR_GRAY2BGR)
        frames_raw.append(frame_bgr)

    mp4_name = "original_raw_ch1.mp4"
    imageio.mimsave(mp4_name, frames_raw, fps=8, format='FFMPEG')
    print(f"Saved: {mp4_name}")
except Exception as e:
    print("MP4 export of original stack failed:", e)

# ==========================================
# 11) EXTRA: Side-by-side (RAW | SEGMENTED) MP4
# ==========================================
try:
    def to_uint8_grayscale(vol):
        vol = vol.astype(np.float32)
        vmin, vmax = float(vol.min()), float(vol.max())
        if vmax <= vmin:
            return (np.zeros_like(vol, dtype=np.uint8))
        return (255.0 * (vol - vmin) / (vmax - vmin)).astype(np.uint8)

    raw_stack_u8 = to_uint8_grayscale(image)      # left panel (raw Ch1)
    seg_base_u8  = to_uint8_grayscale(image_2)    # right base (Ch2)

    Z, H, W = raw_stack_u8.shape
    fps = 8

    out_name = "raw_vs_segmented_side_by_side.mp4"
    writer = imageio.get_writer(out_name, fps=fps, format="FFMPEG")

    for z in range(Z):
        left_bgr = cv2.cvtColor(raw_stack_u8[z], cv2.COLOR_GRAY2BGR)
        base_bgr = cv2.cvtColor(seg_base_u8[z], cv2.COLOR_GRAY2BGR)

        if 'cell_mask' in globals():
            cell = (cell_mask[z].astype(np.uint8) * 255)
            overlay = base_bgr.copy()
            overlay[..., 1] = np.maximum(overlay[..., 1], cell)
            right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
        else:
            right_bgr = base_bgr


        # --- draw ONLY in-cell lysosomes on the right panel ---
        if 'blobs' in globals() and len(blobs) > 0 and 'cell_mask' in globals():
            z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
            for b in z_blobs:
                y, x = int(round(b[1])), int(round(b[2]))
                r = max(2, int(round(b[3])))
                # draw only if this blob center is inside the cell on this slice
                if 0 <= y < H and 0 <= x < W and cell_mask[z, y, x]:
                    cv2.circle(right_bgr, (x, y), r, (255, 255, 0), 2)  # yellow (in-cell only)
        # NOTE: no drawing at all for outside points


        if left_bgr.shape != right_bgr.shape:
            right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                   interpolation=cv2.INTER_NEAREST)
        divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
        frame = cv2.hconcat([left_bgr, divider, right_bgr])

        writer.append_data(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    writer.close()
    print(f"Saved: {out_name}")

except Exception as e:
    try:
        print("FFMPEG writer failed; attempting GIF fallback. Error:", e)
        frames = []
        for z in range(raw_stack_u8.shape[0]):
            left_bgr = cv2.cvtColor(raw_stack_u8[z], cv2.COLOR_GRAY2BGR)
            base_bgr = cv2.cvtColor(seg_base_u8[z], cv2.COLOR_GRAY2BGR)
            if 'cell_mask' in globals():
                cell = (cell_mask[z].astype(np.uint8) * 255)
                overlay = base_bgr.copy()
                overlay[..., 1] = np.maximum(overlay[..., 1], cell)
                right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
            else:
                right_bgr = base_bgr

            # --- draw ONLY in-cell lysosomes on the right panel ---
            if 'blobs' in globals() and len(blobs) > 0 and 'cell_mask' in globals():
                z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
                for b in z_blobs:
                    y, x = int(round(b[1])), int(round(b[2]))
                    r = max(2, int(round(b[3])))
                    # draw only if this blob center is inside the cell on this slice
                    if 0 <= y < H and 0 <= x < W and cell_mask[z, y, x]:
                        cv2.circle(right_bgr, (x, y), r, (255, 255, 0), 2)  # yellow (in-cell only)
            # NOTE: no drawing at all for outside points

            if left_bgr.shape != right_bgr.shape:
                right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                       interpolation=cv2.INTER_NEAREST)
            divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
            frame = cv2.hconcat([left_bgr, divider, right_bgr])
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        imageio.mimsave("raw_vs_segmented_side_by_side.gif", frames, fps=8)
        print("Saved: raw_vs_segmented_side_by_side.gif")
    except Exception as e2:
        print("Side-by-side export failed:", e2)

# ==========================================
# 12) Run viewer
# ==========================================
napari.run()

In [ ]:
####VERSION MEJORADA##################################

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import cv2
import imageio
import xml.etree.ElementTree as ET

import czifile
import tifffile as tiff

from skimage.feature import blob_log, peak_local_max
from skimage.filters import gaussian, threshold_local
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, binary_erosion
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries
from scipy.ndimage import binary_fill_holes, rotate as ndrotate
import napari

# ===============================
# USER CONFIGURATION
# ===============================
# Path to your file (.tif or .czi)
# file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.tif"
# file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/40A_UAS-TMEM192-3xHA x 40A 71G10 MARCM_around 12h - for quantification_3 Airy-CBs_300425.tif"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/TMEM-HA 6h CB airy_3_170722.tif"

# Correct microscope calibration (µm per pixel)
DEFAULT_VX_VY_UM = 0.04   # <-- YOUR TRUE XY SCALE
MAX_REASONABLE_VXY_UM = 0.5  # µm/px sanity limit
MAX_BODY_VOXELS = 0          # 0 = disable the optional body size filter

# ---------------------------
# Helper: refine radii by distance transform + circle fit (sub-pixel)
# ---------------------------
def refine_radii_via_dt(img3d, blobs, win_px=25, bin_method="sauvola"):
    """
    Refine per-blob radius using a local 2D binary mask + sub-pixel circle fit.
    Returns: blobs with columns [z,y,x,r_px] where r_px is continuous (sub-pixel).
    """
    from skimage.filters import threshold_sauvola, threshold_local, threshold_otsu
    from skimage.morphology import remove_small_objects, disk, binary_opening
    from scipy.ndimage import distance_transform_edt as _edt
    from skimage.measure import label as _label

    if blobs is None or len(blobs) == 0:
        return blobs

    Z, H, W = img3d.shape
    out = blobs.copy().astype(np.float32)

    for i, (zc, yc, xc, _) in enumerate(out):
        z = int(round(float(zc)))
        y = int(round(float(yc)))
        x = int(round(float(xc)))
        if not (0 <= z < Z and 0 <= y < H and 0 <= x < W):
            continue

        y1, y2 = max(0, y - win_px), min(H, y + win_px + 1)
        x1, x2 = max(0, x - win_px), min(W, x + win_px + 1)
        if (y2 <= y1) or (x2 <= x1):
            continue

        patch = img3d[z, y1:y2, x1:x2]

        # Adaptive/local binarization
        if bin_method == "sauvola":
            ws = max(21, 2*(win_px//2)+1)
            thr = threshold_sauvola(patch, window_size=ws, k=0.2)
            bw = patch > thr
        elif bin_method == "local":
            ws = max(21, 2*(win_px//2)+1)
            thr = threshold_local(patch, block_size=ws, offset=-0.2*np.std(patch))
            bw = patch > thr
        else:  # 'otsu'
            try:
                thr = threshold_otsu(patch)
                bw = patch > thr
            except ValueError:
                continue

        # light clean-up
        bw = binary_opening(bw, footprint=disk(1))
        bw = remove_small_objects(bw, min_size=3, connectivity=2)

        # Keep only the component that contains (or is nearest to) the center
        yy, xx = y - y1, x - x1
        if not (0 <= yy < bw.shape[0] and 0 <= xx < bw.shape[1]) or not bw[yy, xx]:
            found = False
            for dy in (-1, 0, 1):
                for dx in (-1, 0, 1):
                    yyy, xxx = yy + dy, xx + dx
                    if 0 <= yyy < bw.shape[0] and 0 <= xxx < bw.shape[1] and bw[yyy, xxx]:
                        yy, xx = yyy, xxx
                        found = True
                        break
                if found:
                    break
            if not found:
                continue

        lab = _label(bw, connectivity=2)
        lbl = lab[yy, xx]
        if lbl == 0:
            continue
        bw_obj = (lab == lbl)

        # --- DT-based radius as fallback ---
        dt = _edt(bw_obj)
        r_px_dt = float(dt[yy, xx]) if (0 <= yy < dt.shape[0] and 0 <= xx < dt.shape[1]) else 0.0

        # --- Sub-pixel circle fit (preferred) ---
        bw_u8 = (bw_obj.astype(np.uint8) * 255)
        cnts, _ = cv2.findContours(bw_u8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        r_px_fit = 0.0
        if cnts:
            def contains_center(c):
                return cv2.pointPolygonTest(c, (float(xx), float(yy)), False) >= 0
            chosen = None
            for c in cnts:
                if contains_center(c):
                    chosen = c
                    break
            if chosen is None:
                chosen = max(cnts, key=cv2.contourArea)
            (cx, cy), r = cv2.minEnclosingCircle(chosen)  # float center & radius (pixels)
            r_px_fit = float(r)

        r_px = r_px_fit if r_px_fit > 0 else r_px_dt
        if r_px <= 0:
            continue

        out[i, 3] = r_px  # continuous, sub-pixel-ish

    return out

# ===============================
# Image loading and metadata
# ===============================
def _try_float(x):
    try:
        return float(x)
    except Exception:
        return None

def _parse_ome_xml(xml_text):
    if not xml_text:
        return None
    def grab(attr):
        m = re.search(fr'PhysicalSize{attr}="([\d\.eE+-]+)"', xml_text)
        return float(m.group(1)) if m else None
    return grab("X"), grab("Y"), grab("Z")

def load_any(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext in (".tif", ".tiff"):
        with tiff.TiffFile(file_path) as tf:
            arr = tf.asarray()
            try:
                ome_xml = tf.ome_metadata
            except Exception:
                ome_xml = None
            vx_um = vy_um = vz_um = None
            if ome_xml:
                vx_um, vy_um, vz_um = _parse_ome_xml(ome_xml)
        img = np.squeeze(arr)

        # Supported layouts
        if img.ndim == 4:
            if img.shape[0] == 2:        # (C, Z, Y, X)
                ch1, ch2 = img[0], img[1]
            elif img.shape[1] == 2:      # (Z, C, Y, X)
                ch1, ch2 = img[:, 0], img[:, 1]
            elif img.shape[-1] == 2:     # (Z, Y, X, C)
                ch1, ch2 = img[..., 0], img[..., 1]
            else:
                raise RuntimeError(f"Unexpected TIFF shape: {img.shape}")
        else:
            raise RuntimeError(f"Unexpected TIFF ndim: {img.ndim}")

        return ch1, ch2, (vx_um, vy_um, vz_um), {"type": "tiff"}
    elif ext == ".czi":
        with czifile.CziFile(file_path) as czi:
            img = np.squeeze(czi.asarray())
            meta_xml = czi.metadata()
        if img.ndim < 3:
            raise RuntimeError(f"CZI has unexpected ndim={img.ndim}")
        if img.shape[0] == 2:
            ch1, ch2 = img[0], img[1]
        elif img.shape[1] == 2:
            ch1, ch2 = img[:, 0], img[:, 1]
        else:
            raise RuntimeError(f"Can't auto-detect 2 channels in CZI shape {img.shape}")

        vx_um = vy_um = vz_um = None
        try:
            r = ET.fromstring(meta_xml)
            def _get_um(axis: str):
                v = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}Value")
                u = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}DefaultUnit")
                if v is None:
                    return None
                val = _try_float(v.text)
                unit = (u.text or "").lower() if u is not None else ""
                if val is None:
                    return None
                if unit in ("m", "meter", "metre") or (unit == "" and val < 1e-3):
                    val *= 1e6
                return val
            vx_um, vy_um, vz_um = _get_um("X"), _get_um("Y"), _get_um("Z")
        except Exception:
            pass

        return ch1, ch2, (vx_um, vy_um, vz_um), {"type": "czi"}
    else:
        raise ValueError("Unsupported file format")

# ========= Load image and scales =========
img_ch1, img_ch2, (vx_um, vy_um, vz_um), meta = load_any(file_path)

# Enforce calibration
if vx_um is None or vy_um is None:
    vx_um = vy_um = DEFAULT_VX_VY_UM
if vz_um is None:
    vz_um = vx_um
if vx_um > MAX_REASONABLE_VXY_UM or vy_um > MAX_REASONABLE_VXY_UM:
    raise ValueError(f"XY pixel size too large: (vx, vy)=({vx_um}, {vy_um}) µm/px")

px_um_xy = float(np.sqrt(vx_um * vy_um))
voxel_um3 = vx_um * vy_um * vz_um
print(f"Voxel size (µm): X={vx_um}, Y={vy_um}, Z={vz_um} | XY µm/px={px_um_xy}")

# ===== Aliases used throughout =====
image   = img_ch1       # lysosome channel
image_2 = img_ch2       # neuron / context channel

# ===============================
# Lysosome detection and scaling
# ===============================
image_smooth = gaussian(image, sigma=1.5)

blobs = blob_log(image_smooth, min_sigma=0.8, max_sigma=3.0, num_sigma=4, threshold=0.003, overlap=1)
if len(blobs) > 0:
    # skimage LoG returns sigma; multiply by sqrt(3) to get radius in px
    blobs[:, 3] *= np.sqrt(3)

# Sub-pixel radius refinement
blobs = refine_radii_via_dt(image_smooth, blobs, win_px=25, bin_method="sauvola")

# Convert per-lysosome metrics to physical units (µm)
if blobs is not None and len(blobs) > 0:
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um
    radius_px = blobs[:, 3]
    radius_um = radius_px * px_um_xy
    diameter_um = 2 * radius_um
    volume_um3 = (4.0/3.0) * np.pi * (radius_um ** 3)
else:
    z_um = y_um = x_um = radius_um = diameter_um = volume_um3 = np.array([])

# Build dataframe
df = pd.DataFrame({
    "id": np.arange(1, (0 if blobs is None else len(blobs)) + 1, dtype=int),
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "radius_um": radius_um,
    "diameter_um": diameter_um,
    "volume_um3": volume_um3,
})

# Optional: make a strictly unique radius column for downstream tools/plots
if len(df) > 0:
    # eps so small it won't affect science; just breaks exact ties deterministically
    eps = 1e-6  # µm (1e-6 µm = 0.001 nm)
    df["radius_um_unique"] = df["radius_um"].to_numpy() + eps * (np.arange(len(df)) % 1000)

df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv")

# ===============================
# CH2 segmentation (CELL vs OUTSIDE) using image_2
# ===============================
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=1.5, preserve_range=True)

# Local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=301, offset=-0.2*np.std(R))
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=20000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(10))
neuron_mask = binary_erosion(neuron_mask, ball(7))

# Soma via distance + cleanup
dist = edt(neuron_mask)
cell_min_radius_vox = 1
cell_mask = (dist >= cell_min_radius_vox)
cell_mask &= neuron_mask
cell_mask = binary_opening(cell_mask, ball(5))
cell_mask = binary_closing(cell_mask, ball(10))
cell_mask = binary_fill_holes(cell_mask)

# Label and territories
body_lab = label(cell_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cells (soma).")

if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels:", int(cell_mask.sum()))

# ==========================================
# ADD-ON: optional filter out oversized/undersized cell bodies
# ==========================================
try:
    if isinstance(cell_seg, np.ndarray) and cell_seg.max() > 0 and MAX_BODY_VOXELS > 0:
        counts = np.bincount(cell_seg.ravel().astype(np.int64))
        drop_labels = np.where(counts < 150000)[0]  # example threshold
        drop_labels = drop_labels[drop_labels != 0]
        if drop_labels.size > 0:
            print(f"Filtered {len(drop_labels)} cells. IDs: {drop_labels.tolist()}")
            to_remove = np.isin(cell_seg, drop_labels)
            cell_seg[to_remove] = 0
            cell_mask[to_remove] = False
            pd.DataFrame({
                "filtered_cell_id_ch2": drop_labels,
                "voxel_count": counts[drop_labels]
            }).to_csv("filtered_cells_gt_threshold.csv", index=False)
        else:
            print(f"No cells exceeded threshold; nothing filtered.")
except Exception as e:
    print("Cell size filtering failed:", e)

# ==========================================
# 5) Map lysosomes to (cell/outside) with per-cell IDs
# ==========================================
location_ch2 = []
cell_id_list = []

if (blobs is not None) and (len(blobs) > 0):
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue
        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv")

# Include zero-count cells in counts-by-cell
try:
    if isinstance(cell_seg, np.ndarray) and cell_seg.max() > 0:
        all_cells = pd.DataFrame({"cell_id_ch2": np.arange(1, int(cell_seg.max()) + 1, dtype=int)})
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts_nonzero = (
                df[df["location_ch2"] == "cell"]
                .groupby("cell_id_ch2")
                .size()
                .reset_index(name="count")
            )
        else:
            lys_counts_nonzero = pd.DataFrame(columns=["cell_id_ch2", "count"])
        lys_counts_all = all_cells.merge(lys_counts_nonzero, on="cell_id_ch2", how="left").fillna({"count": 0})
        lys_counts_all["count"] = lys_counts_all["count"].astype(int)
        lys_counts_all.to_csv("lysosome_counts_by_cell.csv", index=False)
        print("Updated: lysosome_counts_by_cell.csv includes 0-lysosome cells.")
except Exception as e:
    print("Could not expand lysosome_counts_by_cell:", e)

# ==========================================
# 5b) Per-cell (Ch2) volumes (µm^3)
# ==========================================
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if isinstance(cell_seg, np.ndarray) and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# ==========================================
# Napari visualization
# ==========================================
viewer = napari.Viewer()
viewer.add_image(image_2, name="Ch2 raw", blending="additive")
viewer.add_image(image,   name="Ch1 raw", blending="additive")

# Add masks/labels after they are computed
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'

    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Lysosome points overlay (show ONLY lysosomes inside cells)
if (blobs is not None) and len(blobs) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    in_cell_mask = (df["location_ch2"].to_numpy() == "cell")
    if in_cell_mask.any():
        blobs_cell = blobs[in_cell_mask, :3]            # z,y,x
        radii_cell = blobs[in_cell_mask, 3] if blobs.shape[1] > 3 else np.ones(np.count_nonzero(in_cell_mask))
        pts = viewer.add_points(
            blobs_cell,
            size=np.clip(radii_cell * 2, 2, None),
            name='Lysosomes (cell only)'
        )
        try:
            pts.face_color = [0.0, 1.0, 1.0, 1.0]       # cyan
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df.loc[in_cell_mask, 'id'].to_numpy(),
                'cell':   df.loc[in_cell_mask, 'cell_id_ch2'].to_numpy(),
                'diameter_um': df.loc[in_cell_mask, 'diameter_um'].to_numpy(),
                'volume_um3':  df.loc[in_cell_mask, 'volume_um3'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
        except Exception:
            pass

# ==========================================
# 7) Quick fused 2D video (optional)
# ==========================================
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)
    
    if (blobs is not None) and len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                cv2.circle(overlay, (x, y), r, (255, 255, 0), 2)  # draw only in-cell lysosomes
    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# 8) Per-cell rotation videos (X, Y, Z axes) – only for cells with lysosomes
# ==========================================
def _to_uint8(vol):
    v = vol.astype(np.float32)
    vmin, vmax = float(v.min()), float(v.max())
    if vmax <= vmin:
        return np.zeros_like(v, dtype=np.uint8)
    return (255.0 * (v - vmin) / (vmax - vmin)).astype(np.uint8)

def _rotate_points(pts_zyx, center_zyx, angle_deg, axis="Y"):
    if pts_zyx.size == 0:
        return pts_zyx
    theta = np.deg2rad(angle_deg)
    c, s = np.cos(theta), np.sin(theta)
    p = pts_zyx - center_zyx
    if axis.upper() == "Y":     # rotate in ZX plane
        z = p[:, 0]*c + p[:, 2]*s
        y = p[:, 1]
        x = -p[:, 0]*s + p[:, 2]*c
    elif axis.upper() == "X":   # rotate in YZ plane
        z = p[:, 0]
        y = p[:, 1]*c - p[:, 2]*s
        x = p[:, 1]*s + p[:, 2]*c
    else:                       # Z axis (rotate in XY plane)
        z = p[:, 0]
        y = p[:, 1]*c + p[:, 2]*s
        x = -p[:, 1]*s + p[:, 2]*c
    return np.stack([z, y, x], axis=1) + center_zyx

def save_cell_spin_videos_allaxes(cell_labels, blobs, df,
                                  out_dir="cell_spin_videos",
                                  angles=tuple(range(0, 360, 10)),
                                  pad=8, fps=12,
                                  draw_point_radius_px=3):
    if not isinstance(cell_labels, np.ndarray) or cell_labels.max() == 0:
        print("No cells to export.")
        return
    if not isinstance(df, pd.DataFrame) or "cell_id_ch2" not in df.columns:
        print("Missing per-blob metadata; cannot filter by lysosome presence.")
        return

    cell_ids_with_lys = sorted(df.loc[
        (df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0),
        "cell_id_ch2"
    ].unique().astype(int))
    if len(cell_ids_with_lys) == 0:
        print("No cells contain lysosomes; no videos exported.")
        return

    os.makedirs(out_dir, exist_ok=True)
    COL_CYAN, COL_GREEN, TXT_COL = (255, 255, 0), (0, 255, 0), (240, 240, 240)
    props = [p for p in regionprops(cell_labels) if p.label in cell_ids_with_lys]
    axes_all = ["X", "Y", "Z"]
    Ztot, Ytot, Xtot = cell_labels.shape
    exported = 0

    for p in props:
        cid = p.label
        minz, miny, minx, maxz, maxy, maxx = p.bbox
        miny, maxy = max(0, miny - pad), min(Ytot, maxy + pad)
        minx, maxx = max(0, minx - pad), min(Xtot, maxx + pad)
        sub_lab = cell_labels[minz:maxz, miny:maxy, minx:maxx]
        if (sub_lab == cid).sum() == 0:
            continue
        subZ, subY, subX = sub_lab.shape
        center_zyx = np.array([(subZ-1)/2, (subY-1)/2, (subX-1)/2], dtype=np.float32)

        sub_blobs = df[(df["cell_id_ch2"] == cid) & (df["location_ch2"] == "cell")]
        if sub_blobs.empty:
            continue
        sel = sub_blobs.index.to_numpy()
        pts_local = np.stack([
            blobs[sel, 0]-minz,
            blobs[sel, 1]-miny,
            blobs[sel, 2]-minx
        ], axis=1).astype(np.float32)

        for axis in axes_all:
            frames = []
            for ang in angles:
                if axis == "Y":
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(0,2), reshape=False, order=0, mode='nearest')
                elif axis == "X":
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(1,0), reshape=False, order=0, mode='nearest')
                else:  # Z
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(1,2), reshape=False, order=0, mode='nearest')

                mask_rot = (rot_lab > 0)
                footprint = mask_rot.max(axis=0).astype(np.uint8)
                frame = np.zeros((subY, subX, 3), dtype=np.uint8)

                if footprint.any():
                    green = (footprint * 255)
                    overlay = frame.copy()
                    overlay[..., 1] = np.maximum(overlay[..., 1], green)
                    frame = cv2.addWeighted(frame, 1.0, overlay, 0.35, 0.0)
                    contours, _ = cv2.findContours((footprint*255), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    if contours:
                        cv2.drawContours(frame, contours, -1, COL_GREEN, 1, lineType=cv2.LINE_AA)

                pts_rot = _rotate_points(pts_local, center_zyx, ang, axis=axis)
                ys = np.clip(np.round(pts_rot[:, 1]).astype(int), 0, subY-1)
                xs = np.clip(np.round(pts_rot[:, 2]).astype(int), 0, subX-1)
                for k in range(len(xs)):
                    cv2.circle(frame, (int(xs[k]), int(ys[k])), draw_point_radius_px, COL_CYAN, -1, lineType=cv2.LINE_AA)

                frames.append(frame)

            out_path = os.path.join(out_dir, f"cell_{cid:03d}_spin{axis}.mp4")
            try:
                imageio.mimsave(out_path, [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames],
                                fps=fps, format='FFMPEG')
                print(f"Saved: {out_path}")
            except TypeError:
                out_gif = out_path.replace(".mp4", ".gif")
                imageio.mimsave(out_gif, [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames], fps=fps)
                print(f"Saved (GIF): {out_gif}")
            exported += 1

    print(f"Exported {exported} per-cell rotation videos (X, Y, Z) → {out_dir}")

# ---- call it ----
save_cell_spin_videos_allaxes(cell_seg, blobs, df,
                              out_dir="cell_spin_videos",
                              angles=tuple(range(0, 360, 10)),
                              pad=8, fps=12)

# ==========================================
# 9) 3D screenshots (XY, YZ, XZ)
# ==========================================
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

viewer.dims.ndisplay = 3
save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# ==========================================
# 10) EXTRA: Export Original stack (Ch1) as MP4
# ==========================================
try:
    raw_stack = np.array(image, dtype=np.float32)
    raw_norm = (255 * (raw_stack - raw_stack.min()) / (raw_stack.ptp() + 1e-8)).astype(np.uint8)

    frames_raw = []
    for z in range(raw_norm.shape[0]):
        frame_gray = raw_norm[z]
        frame_bgr = cv2.cvtColor(frame_gray, cv2.COLOR_GRAY2BGR)
        frames_raw.append(frame_bgr)

    mp4_name = "original_raw_ch1.mp4"
    imageio.mimsave(mp4_name, frames_raw, fps=8, format='FFMPEG')
    print(f"Saved: {mp4_name}")
except Exception as e:
    print("MP4 export of original stack failed:", e)

# ==========================================
# 11) EXTRA: Side-by-side (RAW | SEGMENTED) MP4
# ==========================================
try:
    def to_uint8_grayscale(vol):
        vol = vol.astype(np.float32)
        vmin, vmax = float(vol.min()), float(vol.max())
        if vmax <= vmin:
            return (np.zeros_like(vol, dtype=np.uint8))
        return (255.0 * (vol - vmin) / (vmax - vmin)).astype(np.uint8)

    raw_stack_u8 = to_uint8_grayscale(image)      # left panel (raw Ch1)
    seg_base_u8  = to_uint8_grayscale(image_2)    # right base (Ch2)

    Z, H, W = raw_stack_u8.shape
    fps = 8

    out_name = "raw_vs_segmented_side_by_side.mp4"
    writer = imageio.get_writer(out_name, fps=fps, format="FFMPEG")

    for z in range(Z):
        left_bgr = cv2.cvtColor(raw_stack_u8[z], cv2.COLOR_GRAY2BGR)
        base_bgr = cv2.cvtColor(seg_base_u8[z], cv2.COLOR_GRAY2BGR)

        if 'cell_mask' in globals():
            cell = (cell_mask[z].astype(np.uint8) * 255)
            overlay = base_bgr.copy()
            overlay[..., 1] = np.maximum(overlay[..., 1], cell)
            right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
        else:
            right_bgr = base_bgr

        if 'blobs' in globals() and (blobs is not None) and len(blobs) > 0 and 'cell_mask' in globals():
            z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
            for b in z_blobs:
                y, x = int(round(b[1])), int(round(b[2]))
                r = max(2, int(round(b[3])))
                if 0 <= y < H and 0 <= x < W and cell_mask[z, y, x]:
                    cv2.circle(right_bgr, (x, y), r, (255, 255, 0), 2)  # in-cell only

        if left_bgr.shape != right_bgr.shape:
            right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                   interpolation=cv2.INTER_NEAREST)
        divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
        frame = cv2.hconcat([left_bgr, divider, right_bgr])
        writer.append_data(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    writer.close()
    print(f"Saved: {out_name}")

except Exception as e:
    try:
        print("FFMPEG writer failed; attempting GIF fallback. Error:", e)
        frames = []
        for z in range(raw_stack_u8.shape[0]):
            left_bgr = cv2.cvtColor(raw_stack_u8[z], cv2.COLOR_GRAY2BGR)
            base_bgr = cv2.cvtColor(seg_base_u8[z], cv2.COLOR_GRAY2BGR)
            if 'cell_mask' in globals():
                cell = (cell_mask[z].astype(np.uint8) * 255)
                overlay = base_bgr.copy()
                overlay[..., 1] = np.maximum(overlay[..., 1], cell)
                right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
            else:
                right_bgr = base_bgr

            if 'blobs' in globals() and (blobs is not None) and len(blobs) > 0 and 'cell_mask' in globals():
                z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
                for b in z_blobs:
                    y, x = int(round(b[1])), int(round(b[2]))
                    r = max(2, int(round(b[3])))
                    if 0 <= y < H and 0 <= x < W and cell_mask[z, y, x]:
                        cv2.circle(right_bgr, (x, y), r, (255, 255, 0), 2)

            if left_bgr.shape != right_bgr.shape:
                right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                       interpolation=cv2.INTER_NEAREST)
            divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
            frame = cv2.hconcat([left_bgr, divider, right_bgr])
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        imageio.mimsave("raw_vs_segmented_side_by_side.gif", frames, fps=8)
        print("Saved: raw_vs_segmented_side_by_side.gif")
    except Exception as e2:
        print("Side-by-side export failed:", e2)

# ==========================================
# 12) Run viewer
# ==========================================
napari.run()

In [ ]:
#####versio mejorada##############

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import cv2
import imageio
import xml.etree.ElementTree as ET

import czifile
import tifffile as tiff

from skimage.feature import blob_log, peak_local_max
from skimage.filters import gaussian, threshold_local
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, binary_erosion
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries
from scipy.ndimage import binary_fill_holes, rotate as ndrotate
import napari

# ===============================
# USER CONFIGURATION
# ===============================
# Path to your file (.tif or .czi)
# file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.tif"
# file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/40A_UAS-TMEM192-3xHA x 40A 71G10 MARCM_around 12h - for quantification_3 Airy-CBs_300425.tif"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/TMEM-HA 6h CB airy_3_170722.tif"

# Calibration (µm per pixel)
DEFAULT_VX_VY_UM = 0.04       # your true XY scale
MAX_REASONABLE_VXY_UM = 0.5   # sanity limit
MAX_BODY_VOXELS = 0           # 0 = disable optional soma-size filter

# Biological radius band (micrometers)
R_MIN_UM, R_MAX_UM = 0.08, 0.40   # clamp all radii to this range

# ===============================
# Image loading and metadata
# ===============================
def _try_float(x):
    try:
        return float(x)
    except Exception:
        return None

def _parse_ome_xml(xml_text):
    if not xml_text:
        return None
    def grab(attr):
        m = re.search(fr'PhysicalSize{attr}="([\d\.eE+-]+)"', xml_text)
        return float(m.group(1)) if m else None
    return grab("X"), grab("Y"), grab("Z")

def load_any(path):
    ext = os.path.splitext(path)[1].lower()
    if ext in (".tif", ".tiff"):
        with tiff.TiffFile(path) as tf:
            arr = tf.asarray()
            try:
                ome_xml = tf.ome_metadata
            except Exception:
                ome_xml = None
            vx_um = vy_um = vz_um = None
            if ome_xml:
                vx_um, vy_um, vz_um = _parse_ome_xml(ome_xml)
        img = np.squeeze(arr)

        # Supported layouts
        if img.ndim == 4:
            if img.shape[0] == 2:        # (C, Z, Y, X)
                ch1, ch2 = img[0], img[1]
            elif img.shape[1] == 2:      # (Z, C, Y, X)
                ch1, ch2 = img[:, 0], img[:, 1]
            elif img.shape[-1] == 2:     # (Z, Y, X, C)
                ch1, ch2 = img[..., 0], img[..., 1]
            else:
                raise RuntimeError(f"Unexpected TIFF shape: {img.shape}")
        else:
            raise RuntimeError(f"Unexpected TIFF ndim: {img.ndim}")
        return ch1, ch2, (vx_um, vy_um, vz_um), {"type": "tiff"}

    elif ext == ".czi":
        with czifile.CziFile(path) as czi:
            arr = np.squeeze(czi.asarray())
            meta_xml = czi.metadata()
        if arr.ndim < 3:
            raise RuntimeError(f"CZI has unexpected ndim={arr.ndim}")
        if arr.shape[0] == 2:
            ch1, ch2 = arr[0], arr[1]
        elif arr.shape[1] == 2:
            ch1, ch2 = arr[:, 0], arr[:, 1]
        else:
            raise RuntimeError(f"Can't auto-detect 2 channels in CZI shape {arr.shape}")

        vx_um = vy_um = vz_um = None
        try:
            r = ET.fromstring(meta_xml)
            def _get_um(axis: str):
                v = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}Value")
                u = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}DefaultUnit")
                if v is None:
                    return None
                val = _try_float(v.text)
                unit = (u.text or "").lower() if u is not None else ""
                if val is None:
                    return None
                if unit in ("m", "meter", "metre") or (unit == "" and val < 1e-3):
                    val *= 1e6
                return val
            vx_um, vy_um, vz_um = _get_um("X"), _get_um("Y"), _get_um("Z")
        except Exception:
            pass

        return ch1, ch2, (vx_um, vy_um, vz_um), {"type": "czi"}

    else:
        raise ValueError("Unsupported file format")

# ========= Load image and scales =========
img_ch1, img_ch2, (vx_um, vy_um, vz_um), meta = load_any(file_path)

# Enforce calibration
if vx_um is None or vy_um is None:
    vx_um = vy_um = DEFAULT_VX_VY_UM
if vz_um is None:
    vz_um = vx_um
if vx_um > MAX_REASONABLE_VXY_UM or vy_um > MAX_REASONABLE_VXY_UM:
    raise ValueError(f"XY pixel size too large: (vx, vy)=({vx_um}, {vy_um}) µm/px")

px_um_xy = float(np.sqrt(vx_um * vy_um))
voxel_um3 = vx_um * vy_um * vz_um
print(f"[Scale] X={vx_um:.4f} µm/px, Y={vy_um:.4f} µm/px, Z={vz_um:.4f} µm/px | XY eq={px_um_xy:.4f} µm/px")

# ===== Aliases used throughout =====
image   = img_ch1       # lysosome channel
image_2 = img_ch2       # neuron / context channel

# Convert your biological band to pixels
R_MIN_PX = R_MIN_UM / px_um_xy
R_MAX_PX = R_MAX_UM / px_um_xy

# ---------------------------
# Helper: refine radii (sub-pixel) with watershed split + robust combine
# ---------------------------
def refine_radii_via_dt(img3d, blobs, win_px=None, bin_method="sauvola"):
    """
    For each LoG center, build a local mask, optionally split clusters,
    and estimate radius with three measures: DT, area-equivalent, and
    sub-pixel min-enclosing circle. Take the median and CLIP to [R_MIN_PX, R_MAX_PX].
    """
    from skimage.filters import threshold_sauvola, threshold_local, threshold_otsu
    from skimage.morphology import remove_small_objects, disk, binary_opening
    from scipy.ndimage import distance_transform_edt as _edt
    from skimage.measure import label as _label

    if blobs is None or len(blobs) == 0:
        return blobs

    Z, H, W = img3d.shape
    out = blobs.copy().astype(np.float32)
    # window tied to biological size → smaller window reduces merging
    if win_px is None:
        win_px = int(np.ceil(3 * R_MAX_PX))

    for i, (zc, yc, xc, _) in enumerate(out):
        z = int(round(float(zc)))
        y = int(round(float(yc)))
        x = int(round(float(xc)))
        if not (0 <= z < Z and 0 <= y < H and 0 <= x < W):
            continue

        y1, y2 = max(0, y - win_px), min(H, y + win_px + 1)
        x1, x2 = max(0, x - win_px), min(W, x + win_px + 1)
        if (y2 <= y1) or (x2 <= x1):
            continue

        patch = img3d[z, y1:y2, x1:x2]

        # Binary mask (local/adaptive)
        if bin_method == "sauvola":
            ws = max(21, 2*(win_px//2)+1)
            thr = threshold_sauvola(patch, window_size=ws, k=0.2)
            bw = patch > thr
        elif bin_method == "local":
            ws = max(21, 2*(win_px//2)+1)
            thr = threshold_local(patch, block_size=ws, offset=-0.2*np.std(patch))
            bw = patch > thr
        else:  # 'otsu'
            try:
                thr = threshold_otsu(patch)
                bw = patch > thr
            except ValueError:
                continue

        # light clean-up
        bw = binary_opening(bw, footprint=disk(1))
        bw = remove_small_objects(bw, min_size=3, connectivity=2)

        # ensure we keep component touching the (nearby) center
        yy, xx = y - y1, x - x1
        if not (0 <= yy < bw.shape[0] and 0 <= xx < bw.shape[1]):
            continue
        if not bw[yy, xx]:
            # try 8-neighborhood nudge
            found = False
            for dy in (-1, 0, 1):
                for dx in (-1, 0, 1):
                    yyy, xxx = yy + dy, xx + dx
                    if 0 <= yyy < bw.shape[0] and 0 <= xxx < bw.shape[1] and bw[yyy, xxx]:
                        yy, xx = yyy, xxx
                        found = True
                        break
                if found:
                    break
            if not found:
                continue

        lab = _label(bw, connectivity=2)
        lbl = lab[yy, xx]
        if lbl == 0:
            continue
        bw_obj = (lab == lbl)

        # ---- split clusters via watershed to avoid merged puncta ----
        dt_map = _edt(bw_obj)
        seeds = peak_local_max(dt_map, min_distance=max(1, int(R_MIN_PX)),
                               labels=bw_obj, exclude_border=False)
        if seeds.size > 0:
            markers = np.zeros_like(bw_obj, dtype=int)
            for j, (yy0, xx0) in enumerate(seeds, start=1):
                markers[yy0, xx0] = j
            seg = watershed(-dt_map, markers=markers, mask=bw_obj)
            # keep the segment containing the (possibly adjusted) center
            seg_lbl = seg[yy, xx]
            bw_obj = (seg == seg_lbl)

        # ---- three radius estimates ----
        r_dt = float(dt_map[yy, xx]) if (0 <= yy < dt_map.shape[0] and 0 <= xx < dt_map.shape[1]) else 0.0
        area = float(bw_obj.sum())
        r_eq = np.sqrt(area / np.pi) if area > 0 else 0.0

        bw_u8 = (bw_obj.astype(np.uint8) * 255)
        cnts, _ = cv2.findContours(bw_u8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        r_fit = 0.0
        if cnts:
            def contains_center(c):
                return cv2.pointPolygonTest(c, (float(xx), float(yy)), False) >= 0
            chosen = next((c for c in cnts if contains_center(c)), max(cnts, key=cv2.contourArea))
            (_, _), rr = cv2.minEnclosingCircle(chosen)
            r_fit = float(rr)

        vals = [v for v in (r_dt, r_eq, r_fit) if v > 0]
        if not vals:
            continue

        r_px = float(np.median(vals))
        # ---- clamp to biological band in pixels ----
        r_px = float(np.clip(r_px, R_MIN_PX, R_MAX_PX))
        out[i, 3] = r_px

    return out

# ===============================
# Lysosome detection (LoG) in the right scale band
# ===============================
image_smooth = gaussian(image, sigma=1.5)

# Limit LoG scales to your biological radius band
min_sigma = max(0.5, R_MIN_PX / np.sqrt(3))
max_sigma = max(min_sigma * 1.2, R_MAX_PX / np.sqrt(3))
num_sigma = int(np.clip((max_sigma - min_sigma) * 2.0, 6, 12))

blobs = blob_log(image_smooth,
                 min_sigma=min_sigma,
                 max_sigma=max_sigma,
                 num_sigma=num_sigma,
                 threshold=0.003,
                 overlap=0.5)

if blobs is not None and len(blobs) > 0:
    # skimage LoG returns sigma; multiply by sqrt(3) to get radius in px
    blobs[:, 3] *= np.sqrt(3)
else:
    blobs = np.empty((0, 4), dtype=float)

# Sub-pixel refinement + watershed split + clipping
blobs = refine_radii_via_dt(image_smooth, blobs, win_px=None, bin_method="sauvola")

# Convert to micrometers
if blobs is not None and len(blobs) > 0:
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um
    radius_um = blobs[:, 3] * px_um_xy
    diameter_um = 2.0 * radius_um
    volume_um3 = (4.0/3.0) * np.pi * (radius_um ** 3)
else:
    z_um = y_um = x_um = radius_um = diameter_um = volume_um3 = np.array([])

# Build results dataframe
df = pd.DataFrame({
    "id": np.arange(1, (0 if blobs is None else len(blobs)) + 1, dtype=int),
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "radius_um": radius_um,
    "diameter_um": diameter_um,
    "volume_um3": volume_um3,
})

# QA: flag anything out of band (should be rare due to clamping)
if len(df) > 0:
    outliers = df[(df["radius_um"] < R_MIN_UM) | (df["radius_um"] > R_MAX_UM)]
    if len(outliers):
        outliers.to_csv("lysosome_radius_outliers.csv", index=False)
        print(f"[QA] {len(outliers)} out-of-range radii saved to lysosome_radius_outliers.csv")

# Optional: strictly unique radius column for plotting/tools (does NOT change science)
if len(df) > 0:
    eps = 1e-6  # µm (tiny, far below measurement precision)
    df["radius_um_unique"] = df["radius_um"].to_numpy() + eps * (np.arange(len(df)) % 1000)

df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv")

# ===============================
# CH2 segmentation (CELL vs OUTSIDE)
# ===============================
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
vol = (vol - vmin) / (vmax - vmin + 1e-8)

ch2 = gaussian(vol, sigma=1.5, preserve_range=True)

# Local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=301, offset=-0.2*np.std(R))
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=20000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(10))
neuron_mask = binary_erosion(neuron_mask, ball(7))

# Soma via distance + cleanup
dist = edt(neuron_mask)
cell_min_radius_vox = 1
cell_mask = (dist >= cell_min_radius_vox)
cell_mask &= neuron_mask
cell_mask = binary_opening(cell_mask, ball(5))
cell_mask = binary_closing(cell_mask, ball(10))
cell_mask = binary_fill_holes(cell_mask)

# Label and territories
body_lab = label(cell_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cells (soma).")

if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels:", int(cell_mask.sum()))

# Optional soma size filter
try:
    if isinstance(cell_seg, np.ndarray) and cell_seg.max() > 0 and MAX_BODY_VOXELS > 0:
        counts = np.bincount(cell_seg.ravel().astype(np.int64))
        drop_labels = np.where(counts < 150000)[0]
        drop_labels = drop_labels[drop_labels != 0]
        if drop_labels.size > 0:
            print(f"Filtered {len(drop_labels)} cells. IDs: {drop_labels.tolist()}")
            to_remove = np.isin(cell_seg, drop_labels)
            cell_seg[to_remove] = 0
            cell_mask[to_remove] = False
            pd.DataFrame({
                "filtered_cell_id_ch2": drop_labels,
                "voxel_count": counts[drop_labels]
            }).to_csv("filtered_cells_gt_threshold.csv", index=False)
except Exception as e:
    print("Cell size filtering failed:", e)

# ===============================
# Map lysosomes to (cell/outside)
# ===============================
location_ch2 = []
cell_id_list = []
if (blobs is not None) and (len(blobs) > 0):
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue
        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list
    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)
    print("Saved: lysosomes_with_cell_vs_outside.csv")

    # counts summaries
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)
    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))
    print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv")

# Include zero-count cells in counts-by-cell
try:
    if isinstance(cell_seg, np.ndarray) and cell_seg.max() > 0:
        all_cells = pd.DataFrame({"cell_id_ch2": np.arange(1, int(cell_seg.max()) + 1, dtype=int)})
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts_nonzero = (
                df[df["location_ch2"] == "cell"]
                .groupby("cell_id_ch2")
                .size()
                .reset_index(name="count")
            )
        else:
            lys_counts_nonzero = pd.DataFrame(columns=["cell_id_ch2", "count"])
        lys_counts_all = all_cells.merge(lys_counts_nonzero, on="cell_id_ch2", how="left").fillna({"count": 0})
        lys_counts_all["count"] = lys_counts_all["count"].astype(int)
        lys_counts_all.to_csv("lysosome_counts_by_cell.csv", index=False)
        print("Updated: lysosome_counts_by_cell.csv includes 0-lysosome cells.")
except Exception as e:
    print("Could not expand lysosome_counts_by_cell:", e)

# ===============================
# Per-cell volumes (µm^3)
# ===============================
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if isinstance(cell_seg, np.ndarray) and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# ===============================
# Visualization in napari
# ===============================
viewer = napari.Viewer()
viewer.add_image(image_2, name="Ch2 raw", blending="additive")
viewer.add_image(image,   name="Ch1 raw", blending="additive")

cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'

    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Show in-cell lysosomes only (points)
if (blobs is not None) and len(blobs) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    in_cell_mask = (df["location_ch2"].to_numpy() == "cell")
    if in_cell_mask.any():
        blobs_cell = blobs[in_cell_mask, :3]            # z,y,x
        radii_cell = blobs[in_cell_mask, 3] if blobs.shape[1] > 3 else np.ones(np.count_nonzero(in_cell_mask))
        pts = viewer.add_points(
            blobs_cell,
            size=np.clip(radii_cell * 2, 2, None),
            name='Lysosomes (cell only)'
        )
        try:
            pts.face_color = [0.0, 1.0, 1.0, 1.0]       # cyan
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df.loc[in_cell_mask, 'id'].to_numpy(),
                'cell':   df.loc[in_cell_mask, 'cell_id_ch2'].to_numpy(),
                'diameter_um': df.loc[in_cell_mask, 'diameter_um'].to_numpy(),
                'volume_um3':  df.loc[in_cell_mask, 'volume_um3'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
        except Exception:
            pass

# ===============================
# Optional fused video of Ch2 + cells + in-cell lysosomes
# ===============================
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)
    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if (blobs is not None) and len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                cv2.circle(overlay, (x, y), r, (255, 255, 0), 2)  # draw only in-cell lysosomes
    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# Run viewer
# ==========================================
napari.run()

In [ ]:
#####ultima version##

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import cv2
import imageio
import xml.etree.ElementTree as ET

import czifile
import tifffile as tiff

from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, binary_erosion
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label
from skimage.segmentation import watershed
from scipy.ndimage import binary_fill_holes
import napari

# ===============================
# USER CONFIGURATION
# ===============================
# Path to your file (.tif or .czi)
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.tif"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/40A_UAS-TMEM192-3xHA x 40A 71G10 MARCM_around 12h - for quantification_3 Airy-CBs_300425.tif"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/TMEM-HA 6h CB airy_3_170722.tif"

# Correct microscope calibration (µm per pixel)
DEFAULT_VX_VY_UM = 0.04   # <-- YOUR TRUE XY SCALE
MAX_REASONABLE_VXY_UM = 0.5  # µm/px sanity limit

# ---------------------------
# Helper: refine radii by distance transform
# ---------------------------
def refine_radii_via_dt(img3d, blobs, win_px=25, bin_method="sauvola"):
    from skimage.filters import threshold_sauvola, threshold_local, threshold_otsu
    from skimage.morphology import remove_small_objects, disk, binary_opening
    from scipy.ndimage import distance_transform_edt as _edt
    from skimage.measure import label as _label

    if blobs is None or len(blobs) == 0:
        return blobs

    Z, H, W = img3d.shape
    out = blobs.copy().astype(np.float32)

    for i, (zc, yc, xc, _) in enumerate(out):
        z, y, x = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= z < Z and 0 <= y < H and 0 <= x < W):
            continue

        y1, y2 = max(0, y - win_px), min(H, y + win_px + 1)
        x1, x2 = max(0, x - win_px), min(W, x + win_px + 1)
        patch = img3d[z, y1:y2, x1:x2]

        if bin_method == "sauvola":
            ws = max(21, 2*(win_px//2)+1)
            thr = threshold_sauvola(patch, window_size=ws, k=0.2)
            bw = patch > thr
        elif bin_method == "local":
            ws = max(21, 2*(win_px//2)+1)
            thr = threshold_local(patch, block_size=ws, offset=-0.2*np.std(patch))
            bw = patch > thr
        else:
            try:
                from skimage.filters import threshold_otsu
                thr = threshold_otsu(patch)
                bw = patch > thr
            except ValueError:
                continue

        bw = binary_opening(bw, footprint=disk(1))
        bw = remove_small_objects(bw, min_size=3, connectivity=2)

        yy, xx = y - y1, x - x1
        if not (0 <= yy < bw.shape[0] and 0 <= xx < bw.shape[1]) or not bw[yy, xx]:
            continue

        lab = _label(bw)
        lbl = lab[yy, xx]
        if lbl == 0:
            continue
        bw_obj = (lab == lbl)
        dt = _edt(bw_obj)
        r_px = float(dt[yy, xx])
        if r_px <= 0:
            continue
        out[i, 3] = r_px

    return out

# ===============================
# Image loading and metadata
# ===============================
def _try_float(x):
    try:
        return float(x)
    except:
        return None

def _parse_ome_xml(xml_text):
    if not xml_text:
        return None
    def grab(attr):
        m = re.search(fr'PhysicalSize{attr}="([\d\.eE+-]+)"', xml_text)
        return float(m.group(1)) if m else None
    return grab("X"), grab("Y"), grab("Z")

def load_any(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext in (".tif", ".tiff"):
        with tiff.TiffFile(file_path) as tf:
            arr = tf.asarray()
            try:
                ome_xml = tf.ome_metadata
            except Exception:
                ome_xml = None
            vx_um = vy_um = vz_um = None
            if ome_xml:
                vx_um, vy_um, vz_um = _parse_ome_xml(ome_xml)
        img = np.squeeze(arr)
        if img.ndim == 4:
            ch1, ch2 = (img[0], img[1]) if img.shape[0] == 2 else (img[:, 0], img[:, 1])
        else:
            raise RuntimeError("Unexpected TIFF shape")
        return ch1, ch2, (vx_um, vy_um, vz_um), {"type": "tiff"}
    else:
        raise ValueError("Unsupported file format")

# Load image and scales
img_ch1, img_ch2, (vx_um, vy_um, vz_um), meta = load_any(file_path)
if vx_um is None or vy_um is None:
    vx_um = vy_um = DEFAULT_VX_VY_UM
if vz_um is None:
    vz_um = vx_um

if vx_um > MAX_REASONABLE_VXY_UM:
    raise ValueError(f"XY pixel size too large: {vx_um} µm/px")

px_um_xy = float(np.sqrt(vx_um * vy_um))
voxel_um3 = vx_um * vy_um * vz_um
print(f"Voxel size (µm): X={vx_um}, Y={vy_um}, Z={vz_um}")

# ===== Aliases used throughout (fixes NameError) =====
image   = img_ch1       # lysosome channel
image_2 = img_ch2       # neuron / context channel

# ===============================
# Lysosome detection and scaling
# ===============================
image_smooth = gaussian(image, sigma=1.5)

blobs = blob_log(image_smooth, min_sigma=0.8, max_sigma=3.0, num_sigma=4, threshold=0.003)
if len(blobs) > 0:
    blobs[:, 3] *= np.sqrt(3)

blobs = refine_radii_via_dt(image_smooth, blobs)
if len(blobs) > 0:
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um
    radius_um = blobs[:, 3] * px_um_xy
    diameter_um = 2 * radius_um
    volume_um3 = (4/3) * np.pi * radius_um**3
else:
    z_um = y_um = x_um = radius_um = diameter_um = volume_um3 = np.array([])

df = pd.DataFrame({
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "radius_um": radius_um,
    "diameter_um": diameter_um,
    "volume_um3": volume_um3,
})
df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv")

# ==========================================
# 4) CH2: segmentation (CELL vs OUTSIDE)
# ==========================================
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=1.5, preserve_range=True)

# Local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=301, offset=-0.2*np.std(R))
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=20000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(10))
neuron_mask = binary_erosion(neuron_mask, ball(7))

# Soma via distance + cleanup
dist = edt(neuron_mask)
cell_min_radius_vox = 1
cell_mask = (dist >= cell_min_radius_vox)
cell_mask &= neuron_mask
cell_mask = binary_opening(cell_mask, ball(5))
cell_mask = binary_closing(cell_mask, ball(10))
cell_mask = binary_fill_holes(cell_mask)

# Label and territories
body_lab = label(cell_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cells (soma).")

if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels:", int(cell_mask.sum()))

# ===============================
# 5) Map lysosomes to (cell/outside) with per-cell IDs
# ===============================
location_ch2 = []
cell_id_list = []

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue
        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv (µm-only metrics)")

# Include zero-count cells
try:
    if isinstance(cell_seg, np.ndarray) and cell_seg.max() > 0:
        all_cells = pd.DataFrame({
            "cell_id_ch2": np.arange(1, int(cell_seg.max()) + 1, dtype=int)
        })

        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts_nonzero = (
                df[df["location_ch2"] == "cell"]
                .groupby("cell_id_ch2")
                .size()
                .reset_index(name="count")
            )
        else:
            lys_counts_nonzero = pd.DataFrame(columns=["cell_id_ch2", "count"])

        lys_counts_all = (
            all_cells.merge(lys_counts_nonzero, on="cell_id_ch2", how="left")
                     .fillna({"count": 0})
        )
        lys_counts_all["count"] = lys_counts_all["count"].astype(int)

        lys_counts_all.to_csv("lysosome_counts_by_cell.csv", index=False)
        print("Updated: lysosome_counts_by_cell.csv now includes cells with 0 lysosomes.")
except Exception as e:
    print("Could not expand lysosome_counts_by_cell with zero-count cells:", e)

# ==========================================
# 6) Visualization (now that cell_mask exists)
# ==========================================
viewer = napari.Viewer()
viewer.add_image(img_ch2, name="Ch2 raw")
viewer.add_image(img_ch1, name="Ch1 raw")

# add labels only AFTER cell_mask/cell_seg are defined
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.7)

# ==========================================
# 7) Quick fused 2D video (optional)
# ==========================================
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        # iterate only over blobs that are inside the cell on this slice
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                cv2.circle(overlay, (x, y), r, (255, 255, 0), 2)
    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# 8) Per-cell rotation videos (X, Y, Z axes) – only for cells with lysosomes
# ==========================================
from scipy.ndimage import rotate as ndrotate
from skimage.measure import regionprops  # needed later in helper

def _to_uint8(vol):
    v = vol.astype(np.float32)
    vmin, vmax = float(v.min()), float(v.max())
    if vmax <= vmin:
        return np.zeros_like(v, dtype=np.uint8)
    return (255.0 * (v - vmin) / (vmax - vmin)).astype(np.uint8)

def _rotate_points(pts_zyx, center_zyx, angle_deg, axis="Y"):
    if pts_zyx.size == 0:
        return pts_zyx
    theta = np.deg2rad(angle_deg)
    c, s = np.cos(theta), np.sin(theta)
    p = pts_zyx - center_zyx
    if axis.upper() == "Y":
        z = p[:, 0]*c + p[:, 2]*s
        y = p[:, 1]
        x = -p[:, 0]*s + p[:, 2]*c
    elif axis.upper() == "X":
        z = p[:, 0]
        y = p[:, 1]*c - p[:, 2]*s
        x = p[:, 1]*s + p[:, 2]*c
    else:
        z = p[:, 0]
        y = p[:, 1]*c + p[:, 2]*s
        x = -p[:, 1]*s + p[:, 2]*c
    return np.stack([z, y, x], axis=1) + center_zyx

def save_cell_spin_videos_allaxes(cell_labels, blobs, df,
                                  out_dir="cell_spin_videos",
                                  angles=tuple(range(0, 360, 10)),
                                  pad=8, fps=12,
                                  draw_point_radius_px=3):
    if not isinstance(cell_labels, np.ndarray) or cell_labels.max() == 0:
        print("No cells to export.")
        return
    if not isinstance(df, pd.DataFrame) or "cell_id_ch2" not in df.columns:
        print("Missing per-blob metadata; cannot filter by lysosome presence.")
        return

    cell_ids_with_lys = sorted(df.loc[
        (df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0),
        "cell_id_ch2"
    ].unique().astype(int))
    if len(cell_ids_with_lys) == 0:
        print("No cells contain lysosomes; no videos exported.")
        return

    os.makedirs(out_dir, exist_ok=True)
    COL_CYAN, COL_GREEN, TXT_COL = (255, 255, 0), (0, 255, 0), (240, 240, 240)
    props = [p for p in regionprops(cell_labels) if p.label in cell_ids_with_lys]
    axes_all = ["X", "Y", "Z"]
    Ztot, Ytot, Xtot = cell_labels.shape
    exported = 0

    for p in props:
        cid = p.label
        minz, miny, minx, maxz, maxy, maxx = p.bbox
        miny, maxy = max(0, miny - pad), min(Ytot, maxy + pad)
        minx, maxx = max(0, minx - pad), min(Xtot, maxx + pad)
        sub_lab = cell_labels[minz:maxz, miny:maxy, minx:maxx]
        if (sub_lab == cid).sum() == 0:
            continue
        subZ, subY, subX = sub_lab.shape
        center_zyx = np.array([(subZ-1)/2, (subY-1)/2, (subX-1)/2], dtype=np.float32)

        sub_blobs = df[(df["cell_id_ch2"] == cid) & (df["location_ch2"] == "cell")]
        if sub_blobs.empty:
            continue
        sel = sub_blobs.index.to_numpy()
        pts_local = np.stack([
            blobs[sel, 0]-minz,
            blobs[sel, 1]-miny,
            blobs[sel, 2]-minx
        ], axis=1).astype(np.float32)

        for axis in axes_all:
            frames = []
            for ang in angles:
                if axis == "Y":
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(0,2), reshape=False, order=0, mode='nearest')
                elif axis == "X":
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(1,0), reshape=False, order=0, mode='nearest')
                else:
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(1,2), reshape=False, order=0, mode='nearest')

                mask_rot = (rot_lab > 0)
                footprint = mask_rot.max(axis=0).astype(np.uint8)
                frame = np.zeros((subY, subX, 3), dtype=np.uint8)

                if footprint.any():
                    green = (footprint * 255)
                    overlay = frame.copy()
                    overlay[..., 1] = np.maximum(overlay[..., 1], green)
                    frame = cv2.addWeighted(frame, 1.0, overlay, 0.35, 0.0)
                    contours, _ = cv2.findContours((footprint*255), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    if contours:
                        cv2.drawContours(frame, contours, -1, COL_GREEN, 1, lineType=cv2.LINE_AA)

                pts_rot = _rotate_points(pts_local, center_zyx, ang, axis=axis)
                ys = np.clip(np.round(pts_rot[:, 1]).astype(int), 0, subY-1)
                xs = np.clip(np.round(pts_rot[:, 2]).astype(int), 0, subX-1)
                for k in range(len(xs)):
                    cv2.circle(frame, (int(xs[k]), int(ys[k])), draw_point_radius_px, COL_CYAN, -1, lineType=cv2.LINE_AA)

                frames.append(frame)

            out_path = os.path.join(out_dir, f"cell_{cid:03d}_spin{axis}.mp4")
            try:
                imageio.mimsave(out_path,
                                [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames],
                                fps=fps, format='FFMPEG')
                print(f"Saved: {out_path}")
            except TypeError:
                out_gif = out_path.replace(".mp4", ".gif")
                imageio.mimsave(out_gif,
                                [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames],
                                fps=fps)
                print(f"Saved (GIF): {out_gif}")
            exported += 1

    print(f"Exported {exported} per-cell rotation videos (X, Y, Z) → {out_dir}")

# ---- call it ----
save_cell_spin_videos_allaxes(cell_seg, blobs, df,
                              out_dir="cell_spin_videos",
                              angles=tuple(range(0, 360, 10)),
                              pad=8, fps=12)

# ==========================================
# 9) 3D screenshots (XY, YZ, XZ)
# ==========================================
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

viewer.dims.ndisplay = 3
save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# ==========================================
# 10) EXTRA: Export Original stack (Ch1) as MP4
# ==========================================
try:
    raw_stack = np.array(image, dtype=np.float32)
    raw_norm = (255 * (raw_stack - raw_stack.min()) / (raw_stack.ptp() + 1e-8)).astype(np.uint8)

    frames_raw = []
    for z in range(raw_norm.shape[0]):
        frame_gray = raw_norm[z]
        frame_bgr = cv2.cvtColor(frame_gray, cv2.COLOR_GRAY2BGR)
        frames_raw.append(frame_bgr)

    mp4_name = "original_raw_ch1.mp4"
    imageio.mimsave(mp4_name, frames_raw, fps=8, format='FFMPEG')
    print(f"Saved: {mp4_name}")
except Exception as e:
    print("MP4 export of original stack failed:", e)

# ==========================================
# 11) EXTRA: Side-by-side (RAW | SEGMENTED) MP4
# ==========================================
try:
    def to_uint8_grayscale(vol):
        vol = vol.astype(np.float32)
        vmin, vmax = float(vol.min()), float(vol.max())
        if vmax <= vmin:
            return (np.zeros_like(vol, dtype=np.uint8))
        return (255.0 * (vol - vmin) / (vmax - vmin)).astype(np.uint8)

    raw_stack_u8 = to_uint8_grayscale(image)      # left panel (raw Ch1)
    seg_base_u8  = to_uint8_grayscale(image_2)    # right base (Ch2)

    Z, H, W = raw_stack_u8.shape
    fps = 8

    out_name = "raw_vs_segmented_side_by_side.mp4"
    writer = imageio.get_writer(out_name, fps=fps, format="FFMPEG")

    for z in range(Z):
        left_bgr = cv2.cvtColor(raw_stack_u8[z], cv2.COLOR_GRAY2BGR)
        base_bgr = cv2.cvtColor(seg_base_u8[z], cv2.COLOR_GRAY2BGR)

        if 'cell_mask' in globals():
            cell = (cell_mask[z].astype(np.uint8) * 255)
            overlay = base_bgr.copy()
            overlay[..., 1] = np.maximum(overlay[..., 1], cell)
            right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
        else:
            right_bgr = base_bgr

        if 'blobs' in globals() and len(blobs) > 0 and 'cell_mask' in globals():
            z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
            for b in z_blobs:
                y, x = int(round(b[1])), int(round(b[2]))
                r = max(2, int(round(b[3])))
                if 0 <= y < H and 0 <= x < W and cell_mask[z, y, x]:
                    cv2.circle(right_bgr, (x, y), r, (255, 255, 0), 2)

        if left_bgr.shape != right_bgr.shape:
            right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                   interpolation=cv2.INTER_NEAREST)
        divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
        frame = cv2.hconcat([left_bgr, divider, right_bgr])

        writer.append_data(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    writer.close()
    print(f"Saved: {out_name}")

except Exception as e:
    try:
        print("FFMPEG writer failed; attempting GIF fallback. Error:", e)
        frames = []
        for z in range(raw_stack_u8.shape[0]):
            left_bgr = cv2.cvtColor(raw_stack_u8[z], cv2.COLOR_GRAY2BGR)
            base_bgr = cv2.cvtColor(seg_base_u8[z], cv2.COLOR_GRAY2BGR)
            if 'cell_mask' in globals():
                cell = (cell_mask[z].astype(np.uint8) * 255)
                overlay = base_bgr.copy()
                overlay[..., 1] = np.maximum(overlay[..., 1], cell)
                right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
            else:
                right_bgr = base_bgr

            if 'blobs' in globals() and len(blobs) > 0 and 'cell_mask' in globals():
                z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
                for b in z_blobs:
                    y, x = int(round(b[1])), int(round(b[2]))
                    r = max(2, int(round(b[3])))
                    if 0 <= y < H and 0 <= x < W and cell_mask[z, y, x]:
                        cv2.circle(right_bgr, (x, y), r, (255, 255, 0), 2)

            if left_bgr.shape != right_bgr.shape:
                right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                       interpolation=cv2.INTER_NEAREST)
            divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
            frame = cv2.hconcat([left_bgr, divider, right_bgr])
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        imageio.mimsave("raw_vs_segmented_side_by_side.gif", frames, fps=8)
        print("Saved: raw_vs_segmented_side_by_side.gif")
    except Exception as e2:
        print("Side-by-side export failed:", e2)

# ==========================================
# 12) Run viewer
# ==========================================
napari.run()